In [24]:
from ptlpinns.odes import numerical
import numpy as np
import matplotlib.pyplot as plt
import time

In [25]:
N_ITER = 500

### Define time domain

In [26]:
Nx = 100
x_span = (0, 2) 
Nt = 100
t_span = (0, 5)

x = np.linspace(x_span[0], x_span[1], Nx)
t = np.linspace(t_span[0], t_span[1], Nt)
mesh_x, mesh_t = np.meshgrid(x, t)

### Equation parameters

In [ ]:
# KPP Fisher
epsilons = [0.0]
D = 0.05 

def u_0_function(x):
    return np.sin(np.pi * x / 2)

u_0 = [u_0_function for _ in range(len(epsilons))]  
forcing = [lambda x, t: 0 for _ in range(len(epsilons))] 
bcs = [[lambda t: 0, lambda t: 0] for _ in range(len(epsilons))]  
polynomial = [lambda u: -u + u**2 for _ in range(len(epsilons))]

In [28]:
c = 1.0           
u0 = u_0_function(x)
v0 = np.zeros_like(x)      
U0 = np.concatenate([u0, v0])

In [29]:
time_RK45_KPP, time_Radau_KPP = [], []

for j in range(N_ITER):

    start_RK45 = time.perf_counter()
    numerical.solution_KPP(epsilons, D, polynomial, x_span, t_span, Nx, Nt, u_0, forcing, bcs, method="RK45").squeeze()
    end_RK45 = time.perf_counter()

    start_Radau = time.perf_counter()
    numerical.solution_KPP(epsilons, D, polynomial, x_span, t_span, Nx, Nt, u_0, forcing, bcs, method="Radau").squeeze()
    end_Radau = time.perf_counter()

    time_RK45_KPP.append(end_RK45 - start_RK45)
    time_Radau_KPP.append(end_Radau - start_Radau)

KeyboardInterrupt: 

In [ ]:
time_RK45_wave, time_Radau_wave = [], []

for j in range(N_ITER):

    start_RK45 = time.perf_counter()
    numerical.solution_wave(epsilons, c, polynomial, x_span, t_span, Nx, Nt, U0, bcs, method="RK45").squeeze()
    end_RK45 = time.perf_counter()

    start_Radau = time.perf_counter()
    numerical.solution_wave(epsilons, c, polynomial, x_span, t_span, Nx, Nt, U0, bcs, method="Radau").squeeze()
    end_Radau = time.perf_counter()

    time_RK45_wave.append(end_RK45 - start_RK45)
    time_Radau_wave.append(end_Radau - start_Radau)